In [1]:
# First copy the .env.example file to .env and fill in the values
import os
import sys
from dotenv import load_dotenv

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)
load_dotenv() 

True

In [5]:
# from glob import glob
# import json
# json_files = glob("_test_assets\\basic_info\\*.json")
# for json_file in json_files:
#     with open(json_file, "r+") as f:
#         data = json.load(f)
#         if 'cusips' in data:
#             value = data['cusips']
#             if isinstance(value, list):
#                 continue
#             elif isinstance(value, float):
#                 value = []
#             else:
#                 value = value.split(",")
#             data["cusips"] = value
#         else:
#             data["cusips"] = []
#         f.seek(0)
#         json.dump(data, f, indent=4)
#         f.truncate()


In [28]:
from typing import Any, Dict, List, NamedTuple, Union
from promptflow.connections import CustomStrongTypeConnection
from promptflow.contracts.types import Secret
from promptflow.core import ToolProvider
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_community.graphs.graph_store import GraphStore
import networkx as nx
import sys
class CosmosConnection(CustomStrongTypeConnection):
    """My custom strong type connection.

    :param api_key: The api key.
    :type api_key: Secret
    :param api_base: The api base.
    :type api_base: String
    """
    endpoint: str # wss://<YOUR_ENDPOINT>.gremlin.cosmos.azure.com:443/
    traversal_source: str = "g"
    username: str # /dbs/<YOUR_DATABASE>/colls/<YOUR_COLLECTION_OR_GRAPH>
    access_key: Secret


class KnowledgeTriple(NamedTuple):
    """Knowledge triple in the graph."""

    subject: str
    predicate: str
    object_: str

    @classmethod
    def from_string(cls, triple_string: str) -> "KnowledgeTriple":
        """Create a KnowledgeTriple from a string."""
        subject, predicate, object_ = triple_string.strip().split(", ")
        subject = subject[1:]
        object_ = object_[:-1]
        return cls(subject, predicate, object_)

def parse_triples(knowledge_str: str, triple_delimiter: str = "<|>") -> List[KnowledgeTriple]:
    """Parse knowledge triples from the knowledge string."""
    knowledge_str = knowledge_str.strip()
    if not knowledge_str or knowledge_str == "NONE":
        return []
    triple_strs = knowledge_str.split(triple_delimiter)
    results = []
    for triple_str in triple_strs:
        try:
            kg_triple = KnowledgeTriple.from_string(triple_str)
        except ValueError:
            continue
        results.append(kg_triple)
    return results


def get_entities(entity_str: str) -> List[str]:
    """Extract entities from entity string."""
    if entity_str.strip() == "NONE":
        return []
    else:
        return [w.strip() for w in entity_str.split(",")]

    
# https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/graphs/gremlin_graph.py
class CosmosDBGraphStore(ToolProvider):
    @property
    def get_structured_schema(self) -> Dict[str, Any]:
        return self.structured_schema
    
    def __init__(self, connection: CosmosConnection):
        super().__init__()
        try:
            import asyncio
            import async_timeout
            from gremlin_python.driver import client, serializer
            if sys.platform == "win32":
                asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        except ImportError:
            raise ImportError(
                "Please install gremlin-python, asyncio, and async_timeout first: " "`pip3 install gremlinpython"
            )
        self.connection = connection
        self.client = client.Client(
            url=connection.endpoint,
            traversal_source=connection.traversal_source,
            username=connection.username,
            password=connection.access_key,
            message_serializer=serializer.GraphSONSerializersV2d0(),

        )
        self.schema: str = ""
        self.nxGraph: nx.Graph = nx.Graph()

    def refresh_schema(self) -> None:
        """
        Refreshes the Gremlin graph schema information.
        """
        vertex_schema = self.client.submit("g.V().label().dedup()").all().result()
        edge_schema = self.client.submit("g.E().label().dedup()").all().result()
        vertex_properties = (
            self.client.submit(
                "g.V().group().by(label).by(properties().label().dedup().fold())"
            )
            .all()
            .result()[0]
        )

        self.structured_schema = {
            "vertex_labels": vertex_schema,
            "edge_labels": edge_schema,
            "vertice_props": vertex_properties,
        }

        self.schema = "\n".join(
            [
                "Vertex labels are the following:",
                ",".join(vertex_schema),
                "Edge labes are the following:",
                ",".join(edge_schema),
                f"Vertices have following properties:\n{vertex_properties}",
            ]
        )
    
    def refresh_nx_graph(self) -> nx.Graph:
        """
        Returns the graph as a NetworkX graph.
        """
        vertices = self.client.submit("g.V().valueMap()").all().result()
        edges = self.client.submit("g.E().valueMap()").all().result()
        for vertex in vertices:
            self.nxGraph.add_node(vertex["id"], **vertex)
        for edge in edges:
            self.nxGraph.add_edge(edge["outV"], edge["inV"], **edge)
            
        return self.nxGraph
    
    def query(self, query: str, params: dict = {}) -> List[Dict[str, Any]]:
        q = self.client.submit(query)
        return q.all().result()
    
    # Upsert could be done by mergev instead, but 3.6 version that supports the operator isn't recommended by Cosmos
    def build_vertex_query(self, node: Node, base_traversal: str = "g.") -> str:
        base_query = (
            f"{base_traversal}V().has('id','{node.id}').fold()"
            + f".coalesce(unfold(),addV('{node.type}')"
            + f".property('id','{node.id}')"
            + f".property('type','{node.type}')"
        )
        for key, value in node.properties.items():
            if(key == "description" or key == "text_unit_ids" or key == "document_ids" or key == "community_ids"):
                base_query += f".property(list, '{key}', '{value}')"
            if(isinstance(value, list)):
                for val in value:
                    base_query += f".property(list, '{key}', '{val}')"
            if(isinstance(value, list)):
                for val in value:
                    base_query += f".property(list, '{key}', '{val}')"
            else:
                base_query += f".property('{key}', '{value}')"
        print(base_query)
        return base_query + ")"
    
    def build_edge_query(self, relationship: Relationship) -> str:
        source_query = f".has('id','{relationship.source.id}')"
        target_query = f".has('id','{relationship.target.id}')"
        base_query = f""""g.V(){source_query}.as('a')  
            .V(){target_query}.as('b') 
            .choose(
                __.inE('{relationship.type}').where(outV().as('a')),
                __.identity(),
                __.addE('{relationship.type}').from('a').to('b')
            )        
            """.replace("\n", "").replace("\t", "")
        
        for key, value in relationship.properties.items():
            if(key == "weight"):
                base_query += f".property('{key}', union(value('weight'), constant({value})).sum())" 
            # edges don't support list properties
            # if(key == "description" or key == "text_unit_ids" or key == "document_ids"):
            #     base_query += f".property(list, '{key}', '{value}')" 
            else:
                base_query += f".property('{key}', '{value}')"
        print(f"edge query: {base_query}")
        return base_query

    def upsert_node(self, node: Node, node_cache: dict = {}) -> Node:
        # if properties does not have label, add type as label
        if "label" not in node.properties:
            node.properties["label"] = node.type
        if node.id in node_cache:
            return node_cache[node.id]
        else:
            query = self.build_vertex_query(node)
            _ = self.client.submit(query).all().result()[0]
            node_cache[node.id] = node
            return node

    def add_edge(self, relationship: Relationship) -> Any:
        query = self.build_edge_query(relationship)
        return self.client.submit(query).all().result()
    
    def add_graph(
        self, graph: nx.Graph, include_source: bool = False
    ) -> None:
        """
        Upsert a NetworkX Graph into a Gremlin Graph DB.
        """
        node_cache: Dict[Union[str, int], Node] = {}
        for node in graph.nodes(data=True):
            self.upsert_node(Node(id=node[0], type=node[1]["type"], properties=node[1]), node_cache)

        
        for edge in graph.edges(data=True):
            source = edge[0]
            target = edge[1]
            self.add_edge(
                Relationship(
                    source=Node(id=source, type=graph.nodes[source]["type"]),
                    target=Node(id=target, type=graph.nodes[target]["type"]),
                    type=edge[2]["type"],
                    properties=edge[2],
                )
            )


In [37]:
import pandas as pd
import yfinance as yf
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
from glob import glob
import json
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
import nest_asyncio
from utils.text_helpers import clean_text
import hashlib
nest_asyncio.apply()


# From https://www.sec.gov/files/company_tickers.json
sec_ticker_information = json.load(open("_test_assets\\company_tickers_exchange.json"))
graphDbConnection = CosmosConnection(
  configs={
    "endpoint": os.getenv("COSMOS_GRAPH_ENDPOINT"),
    "username": f"/dbs/{os.getenv("COSMOS_GRAPH_DATABASE_NAME")}/colls/{os.getenv('COSMOS_GRAPH_COLLECTION_NAME')}",
    "traversal_source": "g"
  },
  secrets={
    "access_key": os.getenv("COSMOS_GRAPH_ACCESS_KEY")
  }
)

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass
session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)
## Example on how to get financials for a specific company
def get_financials(ticker: str, cik: str, cusips: list[str], session: Session, output_directory: str = "_test_assets\\basic_info"):
    try:
        output_file = f"{output_directory}\\{ticker}_{cik}.json"
        if os.path.exists(output_file):
            return
        stock = yf.Ticker(ticker, session=session)
        info = stock.info
        info['cik'] = cik
        info['cusips'] = cusips
        with open(f"{output_directory}\\{ticker}_{cik}.json", "w") as jsonfile:
            json.dump(info, jsonfile)
        return info
    except Exception as e:
        print(f"Error getting {ticker}: {e}")
        return {}

def upsert_companies(input_directory: str, file_names: List[str] = [], cik_cusip_ticker_csv_file: str = "_test_assets\\sector_communication\\sector_info.csv"):
    graphClient = CosmosDBGraphStore(connection=graphDbConnection)
    files = [f"{input_directory}/{file_name}" for file_name in file_names]
    sector_df = pd.read_csv(cik_cusip_ticker_csv_file)

    for file_json in files:
        with open(file_json, "r") as jsonfile:
            content = json.loads(jsonfile.read())
            print(f"upserting {file_json}")
        #     cusips = []
        #     tickers = []
        #     for cusip_list, ticker_list in sector_df[sector_df['cik'] == content['cik']][['cusips', 'ticker']].values:
        #         if isinstance(cusip_list, str) and cusip_list != "nan":
        #             cusips.extend(cusip_list.split(","))
        #         tickers.append(ticker_list)
        #     node = graphClient.upsert_node(Node(
        #         id=content['entityName'],
        #         type="Company",
        #         properties={
        #         "cik": content.get("cik"),
        #         "tickers": tickers,
        #         "primaryExchange": content.get("primaryExchange", "UNKNOWN"),
        #         "phone": content.get("phone", "UNKNOWN"),
        #         "names": content.get("entityName", "UNKNOWN"),
        #         "partitionKeyType": "primarySector",
        #         "partitionKey": content.get("sector", "UNKNOWN"),
        #         "primarySector": content.get("sector", "UNKNOWN"),
        #         "primaryIndustry": content.get("industry", "UNKNOWN"),
        #         "longBusinessSummary": clean_text(content.get("longBusinessSummary", "UNKNOWN")),
        #         "address1": clean_text(content.get("address1", "UNKNOWN")),
        #         "city": content.get("city", "UNKNOWN"),
        #         "state": content.get("state", "UNKNOWN"),
        #         "zip": content.get("zip", "UNKNOWN"),
        #         "country": content.get("country", "UNKNOWN"),
        #         "cusips": cusips,
        #         }
        # ))
        # upsert_industry(content, graphClient)
        people = []
        for person in content.get("companyOfficers", []):
            people.append(upsert_people(person, content['entityName'], content['sector'], graphClient))
        print(f"upserted {people}")

def upsert_industry(content: dict[str,str], graphClient: CosmosDBGraphStore):
    if content.get("industry") is None:
        return
    result = graphClient.add_edge(Relationship(
        source=Node(id=content['entityName'], type="Company"),
        target=Node(id=content['industry'], 
                    type="Industry", 
                    properties={
                       "partitionKeyType": "sector", 
                       "partitionKey": content['sector'],
                       "sector": content['sector'],
           }),
        type="IN_INDUSTRY",
        
    ))
    return result
def upsert_people(content: dict[str,str], entity_name: str, sector: str, graphClient: CosmosDBGraphStore):
    if content.get("name") is None:
        return
    person_name = clean_text(content['name'])
    person_node = graphClient.upsert_node(Node(
                    id= f"{entity_name}|{person_name}", 
                    type="Person", 
                    properties={
                       "partitionKeyType": "sector",
                       "partitionKey": sector,
                       "sector": sector,
                       "yearBorn": content.get("yearBorn", 0)
           }))
    result = graphClient.add_edge(Relationship(
        source=Node(id=entity_name, type="Company"),
        target=person_node,
        type="HAS_OFFICER",
        properties={
            "title": clean_text(content['title']),
            "fiscalYear": content.get("fiscalYear", 0),
            "totalPay": content.get("totalPay", 0),
            "excersicedValue": content.get("exercisedValue", 0),
            "unexcersicedValue": content.get("unexercisedValue", 0),        
        }
    ))
    return result
sector_df = pd.read_csv("_test_assets\\sector_communication\\sector_info.csv")
## Generate these files through the get_financials function
file_list = [f"{ticker}_{cik}.json" for ticker,cik in sector_df[['ticker', 'cik']].values]
upsert_companies("_test_assets\\basic_info", file_list)



upserting _test_assets\basic_info/GOOGL_1652044.json
g.V().has('id','Alphabet Inc.|Mr. Sundar  Pichai').fold().coalesce(unfold(),addV('Person').property('id','Alphabet Inc.|Mr. Sundar  Pichai').property('type','Person').property('partitionKeyType', 'sector').property('partitionKey', 'Communication Services').property('sector', 'Communication Services').property('yearBorn', '1973').property('label', 'Person')
edge query: "g.V().has('id','Alphabet Inc.').as('a')              .V().has('id','Alphabet Inc.|Mr. Sundar  Pichai').as('b')             .choose(                __.inE('HAS_OFFICER').where(outV().as('a')),                __.identity(),                __.addE('HAS_OFFICER').from('a').to('b')            )                    .property('title', 'CEO & Director').property('fiscalYear', '2023').property('totalPay', '8802824').property('excersicedValue', '0').property('unexcersicedValue', '0')
g.V().has('id','Alphabet Inc.|Ms. Ruth M. Porat').fold().coalesce(unfold(),addV('Person').propert

In [ ]:
from glob import glob
import json
import nest_asyncio
nest_asyncio.apply()

# Add Form-10k Information
# form13_data = pd.read_csv("_test_assets\\sector_communication\\13F_HR_extracted.csv")
# form10k_data_path = "_test_assets\\sector_communication\\10k-clean"

def upsert_10k(input_directory: str):
    graphClient = CosmosDBGraphStore(connection=graphDbConnection)
    for file_json in glob(f"{input_directory}/*"):
        file_content = json.load(open(file_json))
        cik = file_content['cik']
        company_name = file_content['name']
        primaryExchange = file_content['primaryExchange']
        date = file_content['date']
        cusips = file_content['cusips']
        sequence_number = file_content['accession']
        clean_name = company_name.replace("/", "").replace("\\", "").replace("'", "")
        # edgar files are formated {cik}-{yy}-{sequence}.txt
        accession = f"{cik}-{date[2:4]}-{sequence_number}"
        graphClient.upsert_node(Node(
            id=clean_name,
            type="SEC Form",
            properties={
                "filingDate": date,
                "accession": accession,
                "primaryExchange": primaryExchange,
                "partitionKeyValueType": "cik",
                "partitionKey": cik,
                "names": clean_name
            }
        ))
